# Creating batch to test partial feedback setting

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd
import source.players.bayesian_approximator as ba

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle, randint

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

In [7]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "," + ",".join(["Obs" + str(x) for x in range(len(targets))]) + "," + ",".join(["Feed_prob" + str(x) for x in range(len(targets))]) + ",Feed_type" + "\n" 

In [8]:
def print_row(targets, time_horizon, d, a, p, obs, fprob, ftype, title=None):
    return ",".join([str(i) for i in ([d+"_vs_"+a if not title else title] + [time_horizon] + targets + [d] + [a] + [x if x else "" for x in p] + [o for o in obs] + [fp for fp in fprob] + [ftype]) ])+"\n"

In [9]:
def print_row_unk(targets, time_horizon, d, a, p, obs, fprob, ftype, unk, title=None):
    return ",".join([str(i) for i in ([d+"_vs_"+a if not title else title] + [time_horizon] + targets + [d] + [a] + [x if x else "" for x in p] + [unk] + [o for o in obs] + [fp for fp in fprob] + [ftype]) ])+"\n"

### Creating configurations to save into a csv file

In [89]:
for exp in range(10):
    time_horizon = 1000
    T = 10
    targets = []
    for i in range(int(T/5)):
        targets = targets + util.gen_tar_with_len(5)
    values = tuple((v, v) for v in targets)
    feedback_prob = [1 for i in range(T)]
    observabilities = [1 for i in range(T)]
    feedback_type = "full"
    feedback_type1 = "mab"
    distributions = []
    for i in range(5):
        distributions.append(tuple(util.gen_distr(T)))
    chosen_distr = tuple(util.gen_distr(T))
    g = game.PartialFeedbackGame(values, time_horizon, observabilities, feedback_prob, feedback_type = "full")
    g.attackers = [1]
    g.defenders = [0]
    d = [bm.FB(g, 0)]
    prof = []
    prof_s = []
    prof_u = []
    prof_xs = []
    for (i, distr) in enumerate(distributions):
        prof.append(attackers.StochasticAttacker(g, 1, 1, *distr))
        if i < 5:
            prof_s.append(attackers.StochasticAttacker(g, 1, 1, *distr))
        else:
            prof_s.append(None)
        if i < 1:
            prof_xs.append(attackers.StochasticAttacker(g, 1, 1, *distr))
        else:
            prof_xs.append(None)
        prof_u.append(None)
    #prof.append(attackers.StochasticAttacker(g, 1, 1, *chosen_distr))
    #prof_s.append(attackers.StochasticAttacker(g, 1, 1, *chosen_distr))
    #prof_xs.append(attackers.StochasticAttacker(g, 1, 1, *chosen_distr))
    prof.append(attackers.StackelbergAttacker(g, 1, 1))
    prof_s.append(attackers.StackelbergAttacker(g, 1, 1))
    prof_xs.append(attackers.StackelbergAttacker(g, 1, 1))
    prof_u.append(attackers.StackelbergAttacker(g, 1, 1))
    suqrs = []
    for i in range(4):
        suqrs.append(attackers.SUQR(g,1))
    chosen_a_prof = attackers.SUQR(g,1)
    a = [deepcopy(chosen_a_prof)]
    suqrs.append(chosen_a_prof)
    prof = prof + suqrs
    prof_s = prof_s + suqrs
    prof_xs = prof_xs + suqrs
    prof_u = prof_u + suqrs
    #a = [attackers.StochasticAttacker(g, 1, 1, *chosen_distr)]
    #a = [attackers.StackelbergAttacker(g, 1, 1)]
    g.set_players(d, a, prof)
    prior = (str(random.randint(1,101)) for i in range(T))
    defender =["FB1"]
    profile_strings = []
    profile_strings_s = []
    profile_strings_u = []
    profile_strings_xs = []
    for p in prof:
        profile_strings.append(util.print_adv(p))
    for ps in prof_s:
        if ps:
            profile_strings_s.append(util.print_adv(ps))
        else:
            profile_strings_s.append(None)
    for pxs in prof_xs:
        if pxs:
            profile_strings_xs.append(util.print_adv(pxs))
        else:
            profile_strings_xs.append(None)
    for pu in prof_u:
        if pu:
            profile_strings_u.append(util.print_adv(pu))
        else:
            profile_strings_u.append(None)
    experiments_folder = "../Experiments"
    exp_subfolder = "/c10/10_tar"
    batch_name = str(exp)
    batch_file = batch_name + ".csv"
    batch_path = experiments_folder + exp_subfolder + "/" + batch_file
    with open(batch_path, "w+") as f:
        f.write(print_header(targets, prof))
        #f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings_xs,observabilities,feedback_prob,feedback_type,"FB_1v1"))
        #f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings_s,observabilities,feedback_prob,feedback_type,"FB"))
        #f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings,observabilities,feedback_prob,feedback_type,"FB_multi_prof"))
        #f.write(print_row_unk(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings_u,observabilities,feedback_prob,feedback_type,"fusto1","FFB"))
        #f.write(print_row_unk(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings_u,observabilities,feedback_prob,feedback_type,"busto1","BFB"))
        #f.write(print_row_unk(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings_u,observabilities,feedback_prob,feedback_type,"busto1"+"-"+"-".join(prior),"BFB_rnd_pr"))
        f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings,observabilities,feedback_prob,feedback_type,"Full_feedback"))
        f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings,observabilities,feedback_prob,feedback_type1,"MAB_feedback"))

In [83]:
pd.read_csv(batch_path)

,,,,,,,,,,,,,,,,,,,,,,,,Name,T,0,1,2,3,4,Defender,Attacker,Profile,...,Obs1,Obs2,Obs3,Obs4,Feed_prob0,Feed_prob1,Feed_prob2,Feed_prob3,Feed_prob4,Feed_type
FB,1000,0.829,0.934,0.625,0.61,0.795,FB1,sto1-0.086-0.155-0.255-0.332-0.172,sto1-0.15184815184815187-0.4205794205794206-0.016983016983016987-0.07492507492507493-0.33566433566433573,sto1-0.17682317682317683-0.28471528471528473-0.4135864135864136-0.009990009990009992-0.11488511488511491,sto1-0.18381618381618384-0.17882117882117884-0.04495504495504496-0.16483516483516486-0.42757242757242764,sto1-0.05794205794205795-0.33566433566433573-0.04295704295704296-0.39960039960039967-0.16383616383616387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sto1-0.086-0.155-0.255-0.332-0.172,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,full
FB_multi_prof,1000,0.829,0.934,0.625,0.61,0.795,FB1,sto1-0.086-0.155-0.255-0.332-0.172,sto1-0.18381618381618384-0.17882117882117884-0.04495504495504496-0.16483516483516486-0.42757242757242764,sto1-0.115-0.071-0.545-0.104-0.165,sto1-0.6083916083916084-0.0019980019980019984-0.03696303696303697-0.17682317682317683-0.17582417582417584,sto1-0.112-0.058-0.078-0.356-0.396,sto1-0.3493493493493493-0.4004004004004004-0.11711711711711713-0.12412412412412413-0.009009009009009009,sto1-0.011-0.05-0.921-0.016-0.002,sto1-0.081-0.409-0.046-0.039-0.425,sto1-0.15184815184815187-0.4205794205794206-0.016983016983016987-0.07492507492507493-0.33566433566433573,sto1-0.311-0.338-0.176-0.052-0.123,sto1-0.4679999999999999-0.10499999999999997-0.26499999999999996-0.04899999999999999-0.11299999999999998,sto1-0.17682317682317683-0.28471528471528473-0.4135864135864136-0.009990009990009992-0.11488511488511491,sto1-0.52-0.105-0.074-0.021-0.28,sto1-0.284-0.061-0.133-0.503-0.019,sto1-0.3186813186813187-0.11288711288711291-0.2887112887112887-0.07492507492507493-0.2047952047952048,sto1-0.2552552552552553-0.35235235235235235-0.1021021021021021-0.21121121121121122-0.07907907907907909,sto1-0.10889110889110891-0.1158841158841159-0....,sto1-0.29100000000000004-0.04200000000000001-0...,sto1-0.39360639360639366-0.19680319680319683-0...,sto1-0.02-0.317-0.232-0.231-0.2,sto1-0.618-0.133-0.003-0.119-0.127,sto1-0.267-0.015-0.025-0.619-0.074,sto1-0.05794205794205795-0.33566433566433573-0...,sto1-0.26-0.366-0.088-0.13-0.156,sto1-0.086-0.155-0.255-0.332-0.172,sto1-0.096-0.727-0.056-0.091-0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FFB,1000,0.829,0.934,0.625,0.61,0.795,FB1,sto1-0.086-0.155-0.255-0.332-0.172,fusto1,1,1,1,1,1,1,1,1,1,1,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BFB,1000,0.829,0.934,0.625,0.61,0.795,FB1,sto1-0.086-0.155-0.255-0.332-0.172,busto1,1,1,1,1,1,1,1,1,1,1,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BFB_rnd_pr,1000,0.829,0.934,0.625,0.61,0.795,FB1,sto1-0.086-0.155-0.255-0.332-0.172,busto1-,1,1,1,1,1,1,1,1,1,1,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
